In [70]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import calitp_data_analysis.magics

from rt_analysis import rt_filter_map_plot
import build_speedmaps_index

from IPython.display import display, Markdown
import pandas as pd
import geopandas as gpd
from siuba import *

import shared_utils
import segment_speed_utils

In [71]:
catalog = shared_utils.catalog_utils.get_catalog('gtfs_analytics_data')

In [72]:
## parameters cell
itp_id = 182

In [73]:
analysis_date = build_speedmaps_index.ANALYSIS_DATE
import datetime as dt
# if itp_id == 293:
#     analysis_date = dt.date(2023, 9, 13)

In [74]:
speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_itp_id == itp_id)
# speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_name.str.contains('Torr'))

In [75]:
speedmap_index

,organization_name,organization_itp_id,organization_source_record_id,caltrans_district,_is_current,analysis_date,schedule_gtfs_dataset_key,status
42,Los Angeles County Metropolitan Transportation...,182,recPnGkwdpnr8jmHB,07 - Los Angeles,True,2024-10-16,0666caf3ec1ecc96b74f4477ee4bc939,speedmap_segs_available


In [76]:
path = f'{catalog.speedmap_segments.dir}{catalog.speedmap_segments.shape_stop_single_segment_detail}_{analysis_date}.parquet'

In [77]:
speedmap_segs = gpd.read_parquet(path) #  aggregated

In [78]:
# speedmap_segs.columns

In [79]:
speedmap_segs = speedmap_segs >> filter(_.schedule_gtfs_dataset_key == speedmap_index.schedule_gtfs_dataset_key.iloc[0])

In [80]:
speedmap_segs >> head(3)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,route_id,direction_id,stop_pair,stop_pair_name,segment_id,time_of_day,p50_mph,...,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry,n_trips_sch,trips_hr_sch,route_short_name
5265,0666caf3ec1ecc96b74f4477ee4bc939,1d801329ec1c7b1da3460ede3c0fe8bc,1200106_JUNE24,120-13183,1.0,16644__14345,Whittwood Center__Santa Gertrudes / Starbuck,16644-14345-1,AM Peak,14.49,...,16.38,LA Metro Bus Schedule,07 - Los Angeles,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"LINESTRING (-117.99579 33.94440, -117.99579 33...",4,1.333333,120
5266,0666caf3ec1ecc96b74f4477ee4bc939,1d801329ec1c7b1da3460ede3c0fe8bc,1200106_JUNE24,120-13183,1.0,14345__14346,Santa Gertrudes / Starbuck__Santa Gertrudes / ...,14345-14346-1,AM Peak,26.67,...,28.96,LA Metro Bus Schedule,07 - Los Angeles,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"LINESTRING (-117.99307 33.93974, -117.99307 33...",4,1.333333,120
5267,0666caf3ec1ecc96b74f4477ee4bc939,1d801329ec1c7b1da3460ede3c0fe8bc,1200106_JUNE24,120-13183,1.0,14346__14344,Santa Gertrudes / Yermo__Santa Gertrudes / Lef...,14346-14344-1,AM Peak,11.10,...,16.04,LA Metro Bus Schedule,07 - Los Angeles,recPnGkwdpnr8jmHB,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,"LINESTRING (-117.99308 33.93630, -117.99308 33...",4,1.333333,120


In [81]:
peak_test = speedmap_segs >> filter(_.time_of_day == 'AM Peak')

In [82]:
def prep_gdf(gdf):
    
    gdf = gdf.to_crs(calitp_data_analysis.geography_utils.CA_NAD83Albers)

    ## shift to right side of road to display direction
    gdf.geometry = gdf.geometry.apply(shared_utils.rt_utils.try_parallel)
    gdf = gdf.apply(shared_utils.rt_utils.arrowize_by_frequency, axis=1, frequency_col='trips_hr_sch')

    gdf = gdf >> arrange(_.trips_hr_sch)

    return gdf

In [ ]:
peak_test = prep_gdf(peak_test)

In [ ]:
cmap = shared_utils.rt_utils.ACCESS_ZERO_THIRTY_COLORSCALE
url = shared_utils.rt_utils.ACCESS_SPEEDMAP_LEGEND_URL

In [ ]:
peak_test >> head(2)

In [ ]:
peak_test.columns

## export map

In [ ]:
speedmap_state = shared_utils.rt_utils.set_state_export(
                                    peak_test, filename=f'{itp_id}_new', map_type='speedmap',
                                    color_col='p20_mph', cmap=cmap, legend_url=url,
                                    cache_seconds=0, map_title=f'Speedmap Segs {time_of_day} {analysis_date}')

In [ ]:
speedmap_state

## other test

In [60]:
time_of_day = 'PM Peak'

In [61]:
period_speeds = speedmap_segs >> filter(_.time_of_day == time_of_day)

In [62]:
period_test = prep_gdf(period_speeds)

In [63]:
time_of_day_lower = time_of_day.lower().replace(' ', '_')

In [65]:
speedmap_state = shared_utils.rt_utils.set_state_export(
                                    period_test, filename=f'{itp_id}_{time_of_day_lower}_new', map_type='speedmap',
                                    color_col='p20_mph', cmap=cmap, legend_url=url,
                                    cache_seconds=0, map_title=f'Speedmap Segs {time_of_day} {analysis_date}')

writing to calitp-map-tiles/testing/182_pm_peak_new.geojson.gz


In [66]:
speedmap_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Speedmap Segs PM Peak 2024-10-16',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/182_pm_peak_new.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (34.04804074656423, -118.29635317134563),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend_color_access.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTcGVlZG1hcCBTZWdzIFBNIFBlYWsgMjAyNC0xMC0xNiIsICJ1cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2NhbGl0cC1tYXAtdGlsZXMvdGVzdGluZy8xODJfcG1fcGVha19uZXcuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41LCAidG9vbHRpcF9zcGVlZF9rZXkiOiAicDIwX21waCJ9LCAidHlwZSI6ICJzcGVlZG1hcCJ9XSwgImxhdF9sb24iOiBbMzQuMDQ4MDQwNzQ2NTY0MjMsIC0xMTguMjk